## 発表内容

最近日本語対応したAmazon KendraはAmazon S3やAmazon RDS、Salesforceといったリポジトリのコンテンツを一元管理されたインデックスに簡単に集約できます。これにより企業の様々なデータを素早く検索できます。そして、Amazon Comprehendはドキュメント内のテキストの解析を行うことができます。この2つのサービスを組み合わせた文書検索と検索された文書の解析についてデモを交えてご紹介します。

----

## 概要

- Wikipediaの文章をComprehendで分析させてみる（図②）
- KendraにWikipediaの文章を取り込んで日本語検索してみる（図③、④、⑤）

この二つのサービスを組み合わせることで、Comprehendで文書から抽出したキーワードをKendraの絞り込み検索の時に使うといったような応用も可能です。

----

## アーキテクチャ

![](./architecture_kendra_comprehend.png)

----
![](./kendra_flow.png)

----

## デモの流れ

2. Wikipediaから取得したファイルをディレクトリに配置する
3. Kendraインデックスとデータソースコネクタをプロビジョニングします
4. Comprehendを用いたエンティティ抽出（図②）
5. S3へのデータ配置（図③）
6. データソースのsync（図④）
7. 検索！（図⑤）

## 2. Wikipediaから取得したファイルをディレクトリに配置する


In [1]:
!mkdir -p Data Meta && mkdir -p Meta/Data && echo 0 > .metalog

!cat aws.txt && echo '----------'
# aws.txtに書かれた情報をWikipediaからcurlしてくる
!sh ./getpages.sh

Amazon_Web_Services
Amazon.com
Amazon_Simple_Queue_Service
Amazon_Elastic_Compute_Cloud
Amazon_Simple_Storage_Service
----------
Getting https://ja.wikipedia.org/wiki/Amazon_Web_Services
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   861k      0 --:--:-- --:--:-- --:--:--  865k
Getting https://ja.wikipedia.org/wiki/Amazon.com
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  492k  100  492k    0     0  2433k      0 --:--:-- --:--:-- --:--:-- 2439k
Getting https://ja.wikipedia.org/wiki/Amazon_Simple_Queue_Service
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 72432  100 72432    0     0   507k      0 --:--:-- --:--:-- --

In [2]:
# 下準備

# バケット作成
!sh mb.sh

comprehendlab-akazawt-20211118-26244
make_bucket: comprehendlab-akazawt-20211118-26244


## 3. Kendraインデックスとデータソースコネクタをプロビジョニングします

新しい Kendraインデックス の作成には約15分ほどかかります。（3分クッキング方式でデモ実施します）  
デモでは画面上でインデックスとデータソースを作成する手順をご説明します。  

## 4. Comprehendを用いたエンティティ抽出（図②）

Comprehendのデモ  

[Wikipediaの文章](https://ja.wikipedia.org/wiki/Amazon_Web_Services)を取り込んでみる。 コンソールは[こちら](https://console.aws.amazon.com/comprehend/v2/home?region=us-east-1#home)

----

Wikipediaから取得してきたファイルを対象として、SDKを使ってComprehendでエンティティ抽出を行ってみる。

ファイルを逐次読み込み -> Comprehendのdetect_entitiesメソッドでエンティティを取得 -> 整形してjson化してファイルに格納

```python
# ファイル読み込み
    doc_text = open(fname, 'r', encoding="utf-8").read()
...
# detect_entitiesメソッドでエンティティを取得
    for i in range(0, len(doc_text), compre_text_size):
        try:
            entities = client.detect_entities(Text=doc_text[i:i+compre_text_size], LanguageCode='ja')
...
# json化
    attributes["Attributes"] = metadata
    print(json.dumps(attributes, sort_keys=True, indent=4))
```

In [3]:
# 1ファイルでエンティティデータ抽出をスクリプトで試してみる
#  元のWikipediaページ
#    https://ja.wikipedia.org/wiki/Amazon_Web_Services

!python3 getpagemetadata.py Data/Amazon_Web_Services.html

{
    "Attributes": {
        "COMMERCIAL_ITEM": [
            "Route",
            "EC2",
            "Prime Video",
            "Kindle",
            "Echo",
            "Kindle Fire",
            "Fire",
            "Fire HD 10",
            "Fire TV",
            "Prime Air"
        ],
        "DATE": [
            "2月",
            "3月",
            "4月",
            "5月",
            "6月",
            "7月",
            "8月",
            "2006年",
            "2004年",
            "2021年9月2日"
        ],
        "EVENT": [],
        "LOCATION": [
            "アメリカ東部",
            "北バージニア",
            "アメリカ西部",
            "カリフォルニア",
            "サンパウロ",
            "アイルランド",
            "ロンドン",
            "シンガポール",
            "大阪",
            "香港"
        ],
        "ORGANIZATION": [
            "Amazon Web Services",
            "Wikipedia",
            "Amazon_Web_Services",
            "Amazon.com",
            "アマゾン",
            "AWS",
            "Amazon",
            "日本政府

## 5. S3へのデータ配置（図③）

In [7]:
# 今回のデモではS3をデータソースとしてKendraで検索をしたい。そのためにS3にWikipediaの情報をアップロードする。  

!sh upload.sh > /dev/null

## データソースのsyncと検索（図④、⑤）

https://console.aws.amazon.com/kendra/home?region=us-east-1#indexes